https://www.pyimagesearch.com/2019/03/04/holistically-nested-edge-detection-with-opencv-and-deep-learning/

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import glob
import os

import cv2
from keras.preprocessing.image import load_img, save_img
from keras.utils import get_file
import numpy as np

import config

In [ ]:
# change current directory to project root
current_dir = os.getcwd().split('/')[-1]
if current_dir != 'pix2pix-edges-with-color': 
    %cd '..'

### Download Zappos50K dataset

In [ ]:
get_file(f'{config.ZAPPOS_DATASET_NAME}.zip', config.ZAPPOS_DATASET_URL, extract=True, 
         cache_dir='.', cache_subdir='')

### Download HED pretrained model

In [ ]:
get_file(config.HED_MODEL_FILENAME, config.HED_MODEL_URL, cache_dir=config.HED_MODEL_DIR, cache_subdir='')

### HED model custom crop layer

In [ ]:
class CropLayer(object):
    def __init__(self, params, blobs):
        # initialize our starting and ending (x, y) coordinates of the crop
        self.startX = 0
        self.startY = 0
        self.endX = 0
        self.endY = 0
        
    def getMemoryShapes(self, inputs):
        # the crop layer will receive two inputs -- we need to crop
        # the first input blob to match the shape of the second one,
        # keeping the batch size and number of channels
        (inputShape, targetShape) = (inputs[0], inputs[1])
        (batchSize, numChannels) = (inputShape[0], inputShape[1])
        (H, W) = (targetShape[2], targetShape[3])

        # compute the starting and ending crop coordinates
        self.startX = int((inputShape[3] - targetShape[3]) / 2)
        self.startY = int((inputShape[2] - targetShape[2]) / 2)
        self.endX = self.startX + W
        self.endY = self.startY + H

        # return the shape of the volume (we'll perform the actual
        # crop during the forward pass
        return [[batchSize, numChannels, H, W]]

    def forward(self, inputs):
        # use the derived (x, y)-coordinates to perform the crop
        return [inputs[0][:, :, self.startY:self.endY,
                self.startX:self.endX]]

In [ ]:
def detect_edges(img, hed_model):
    """Edges in image are detected using HED model. 
    
    Returns:
        3-dimensional grayscale image of detected edges (white background, black foreground).
    """
    blob = cv2.dnn.blobFromImage(img, mean=(104.00698793, 116.66876762, 122.67891434))
    hed_model.setInput(blob)
    img_edges = hed_model.forward()
    img_edges = img_edges[0, 0]
    img_edges = (255 * img_edges).astype("uint8")
    img_edges = cv2.bitwise_not(img_edges) # invert black/white
    
    # 2D grayscale -> 3D
    img_edges = np.expand_dims(img_edges, axis=-1)
    img_edges = np.repeat(img_edges, config.IMG_N_CHANNELS, axis=-1)
    
    return img_edges

def get_shoe_model_id_to_filepaths_dict(data_dir):
    """The naming convention for each image in the Zappos dataset is SHOE_MODEL_ID.SHOE_COLOR_PATTERN_ID.jpg.
    Only images with a minimum frequency of two shoe model ids  are kept. 

    e.g. For the list of image names below, image 2.1.jpg will not be kept since the shoe model id only occurs once.
    1.1.jpg
    1.2.jpg
    2.1.jpg
    """
    model_id_to_filepaths = {}
    for filepath in glob.glob(f'{data_dir}/*.jpg'):
        filename = os.path.basename(filepath)
        model_id, pattern_id = filename.split('.')[:2]
        filepaths = model_id_to_filepaths.get(model_id, [])
        filepaths.append(filepath)
        model_id_to_filepaths[model_id] = filepaths
    
    model_id_to_filepaths = {model_id: filepaths 
                             for model_id, filepaths in model_id_to_filepaths.items()
                             if len(filepaths) >= config.ZAPPOS_DATASET_MIN_SHOE_MODEL_ID_COUNT}
    
    return model_id_to_filepaths

### Load HED model

In [ ]:
cv2.dnn_registerLayer("Crop", CropLayer)
hed_model = cv2.dnn.readNetFromCaffe(config.HED_MODEL_PROTOTXT_FILEPATH, config.HED_MODEL_FILEPATH)

### Create source (edge) images and copy target images

In [ ]:
data_dir = f'{config.ZAPPOS_DATASET_SNEAKERS_DIR}/Nike'
model_id_to_filepaths = get_shoe_model_id_to_filepaths_dict(data_dir)

for filepaths in model_id_to_filepaths.values():
    img_first = load_img(filepaths[0], target_size=(config.IMG_HEIGHT, config.IMG_WIDTH))
    img_first = np.array(img_first)
    img_edges = detect_edges(img_first, hed_model)
    
    for filepath in filepaths:
        filename = os.path.basename(filepath)
        
        img_source_filepath = os.path.join(config.TRAINING_SOURCE_DIR, filename)
        save_img(img_source_filepath, img_edges)
        
        img_target = load_img(filepath, target_size=(config.IMG_HEIGHT, config.IMG_WIDTH))
        img_target = np.array(img_target)
        img_target_filepath = os.path.join(config.TRAINING_TARGET_DIR, filename)
        save_img(img_target_filepath, img_target)